# Distributed Computation on Many Machines

## This notebook will not run in Binder!

Binder + Dask do not play well (because of `conda`).  You can read this notebook on Binder - to run it you will need to `$ pip install -r requirements-coiled.txt` and run locally.

## Outcomes

- overview of options for distributed compute in Python in 2022,
- demonstration of a AWS/Dask/Coiled/Prefect stack to distribute compute over a cluster on EC2.


## Why distribute compute over many machines?

Limit on the size of a single machine (largest instance on EC2 etc).

Many small machines can be cheaper & larger than the largest single machine.

Modern distributed compute platforms/environments will be fault tolerant to failures of individual workers - a single EC2 instance won't be.

## Ecosystems

Spark:

- accessing Scala code with Python bindings,
- Databricks is a modern way to run Spark.

[Ray](https://docs.ray.io/en/latest/index.html) & [Dask](https://docs.dask.org/en/stable/):

- distributed compute frameworks,
- DAGs for computation.

Tensorflow & PyTorch:

- multi-GPU training,
- accessing C++ code with Python bindings.

Plus more - Celery, lots of AWS Lambda...


## Our focus

A stack of Dask / Coiled / Prefect / EC2.

Requires two accounts - AWS account, Coiled account - Prefect account is optional. 


## Dask

Dask is an execution framework - one scheduler is responsible for executing many workers on many tasks.

<center><img src="../assets/dask.png" alt="Drawing" style="width: 600px;"/></center>

While Dask is a core part of this stack (it gives us concurrent computation - both parallelism + async), we will not write any low level Dask (or Dask DataFrame) code.


## Coiled

<center><img src="../assets/many-machine/f1.png" alt="Drawing" style="width: 600px;"/></center>

Manages AWS infrastructure for running Dask clusters on EC2:

- turns a `requirements.txt` into a *software environment* - Docker image with `pip install`,


## Prefect

Acts as a wrapper around Dask.  Prefect offers more functionality than just Dask execution:

- scheduling,
- monitoring,
- intelligent re-execution of pipelines (aka back-filling).

Prefect 2.0 is currently in beta (not yet production ready) - we are using Prefect 2.0.


# Prefect & Dask on a Single Machine

Let's start by writing the program from the last exercise of the previous notebook:

In [1]:
%%timeit -n 1 -r 1
!python ../src/naive.py

13.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Now try with naive Prefect:

In [2]:
%%timeit -n 1 -r 1
!python ../src/naive_dask_prefect.py

00:38:31.426 | INFO    | prefect.engine - Created flow run 'dangerous-dogfish' for flow 'main'
00:38:31.427 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`
00:38:33.326 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://127.0.0.1:8787/status
00:38:35.195 | INFO    | Flow run 'dangerous-dogfish' -  downloading http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2021/MMSDM_2021_01/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_DVD_DISPATCH_UNIT_SCADA_202101010000.zip
00:38:35.539 | INFO    | Flow run 'dangerous-dogfish' - Created task run 'download-ccd6cdb6-0' for task 'download'
00:38:35.996 | INFO    | Flow run 'dangerous-dogfish' - Submitted task run 'download-ccd6cdb6-0' for execution.
00:38:35.997 | INFO    | Flow run 'dangerous-dogfish' -  processing http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2021/MMSDM_2021_01/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_D

Now let's use Prefect with `asyncio`:

In [3]:
%%timeit -n 1 -r 1
!python ../src/async_prefect.py

00:38:49.126 | INFO    | prefect.engine - Created flow run 'unnatural-mole' for flow 'main'
00:38:49.128 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`
00:38:50.977 | INFO    | prefect.task_runner.dask - The Dask dashboard is available at http://127.0.0.1:8787/status
00:38:52.817 | INFO    | Flow run 'unnatural-mole' -  downloading http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2021/MMSDM_2021_01/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_DVD_DISPATCH_UNIT_SCADA_202101010000.zip
00:38:53.117 | INFO    | Flow run 'unnatural-mole' - Created task run 'download-ccd6cdb6-0' for task 'download'
00:38:53.595 | INFO    | Flow run 'unnatural-mole' - Submitted task run 'download-ccd6cdb6-0' for execution.
00:38:53.595 | INFO    | Flow run 'unnatural-mole' -  processing http://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/2021/MMSDM_2021_01/MMSDM_Historical_Data_SQLLoader/DATA/PUBLIC_DVD_DISPATCH_UNI

# Prefect & Dask Running on a Coiled Cluster (Many Machines)

<center><img src="../assets/many-machine/f2.png" alt="Drawing" style="width: 600px;"/></center>

Requires a few accounts to get setup:

- AWS account - cluster will run on EC2,
- Coiled account - adds & manages AWS infrastructure needed for a Dask cluster.

Stack:

- EC2,
- Dask,
- Prefect,
- Coiled.

Example of running on a Coiled cluster:

In [4]:
%%timeit -n 1 -r 1
!python ../src/dask_coiled_prefect.py

Creating new software environment
Creating new ecr build
STEP 1: FROM coiled/default:sha-6b4e896
STEP 2: COPY environment.yml environment.yml
--> Using cache 715f431717389d9e2bbe2078c5d019fd29c5293c30a365b9bf87ea9ad21e9f3e
--> 715f4317173
--> Using cache 661e893e3cae54adf001e0665a01de463f3564fe632ee5ea91f90b696d835131
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
--> 661e893e3ca
--> Using cache 8e5159d1bd97f7cfb7cb455f2c934ec4c74139efa8d1d34c111818b03c89cd00
STEP 4: ENV PATH /opt/conda/envs/coiled/bin:$PATH
--> 8e5159d1bd9
--> Using cache 539154a40629d3406444a41a4ee44d84aeb160dfe9f86333537c67f3f23a18f0
STEP 5: SHELL ["conda", "run", "-n", "coiled", "/bin/bash", "-c"]
--> 539154a4062
STEP 6: COPY requirements.txt requirements.txt
--> Using cache 7e09dc82cfddb980d4e69456f9fda918eebac3759268b1d20c2f8bad2c2ce2d7
--> 7e09dc82cfd
--> Using cache c22eaadebe87160f8e50cda553911c479

# Setting up the AWS/Dask/Coiled/Prefect stack

## AWS Setup

Pre-requisite is an AWS account.

First setup a new IAM user (below I call this user `coiled`) with programmatic access (key + secret key) - remember to download / copy your credentials to CSV!

We will use this user to manage & run the Coiled cluster on EC2.

Create IAM policies & AWS infrastructure so you can run Dask clusters in your AWS account.

[Coiled AWS setup](https://docs.coiled.io/user_guide/aws-cli.html). 

[Coiled IAM policies](https://docs.coiled.io/user_guide/aws_reference.html) - one is for setting up the IAM user (don't need if you are using credentials with admin access)

- create 2 IAM policies `coiled-setup` & `coiled-ongoing` from JSON,
- attach policies to your IAM user


## Coiled account setup

Create Coiled account - https://cloud.coiled.io/signup - add your credentials in *Cloud Provider*.

Or do the same thing via the shell - create Coiled API token https://cloud.coiled.io/profile:

<center><img src="../assets/many-machine/f3.png" alt="Drawing" style="width: 600px;"/></center>

```shell
$ pip install coiled
#  use token here
$ coiled login
$ coiled setup aws
```

Wasn't sure how to configure `region` with the browser *Cloud Provider*.

Now you can run the Dask example:

In [5]:
!python ../src/dask_coiled.py

/Users/adam/.pyenv/versions/kiwi-pycon-talk/lib/python3.8/site-packages/distributed/client.py:1309: VersionMismatchWarning: Mismatched versions found

+-------------+----------------+----------------+----------------+
| Package     | client         | scheduler      | workers        |
+-------------+----------------+----------------+----------------+
| dask        | 2022.8.0       | 2022.6.0       | 2022.6.0       |
| distributed | 2022.8.0       | 2022.6.0       | 2022.6.0       |
| lz4         | None           | 4.0.0          | 4.0.0          |
| msgpack     | 1.0.4          | 1.0.3          | 1.0.3          |
| numpy       | 1.23.2         | 1.21.6         | 1.21.6         |
| pandas      | 1.4.3          | 1.4.2          | 1.4.2          |
| python      | 3.8.13.final.0 | 3.9.13.final.0 | 3.9.13.final.0 |
| toolz       | 0.12.0         | 0.11.2         | 0.11.2         |
+-------------+----------------+----------------+----------------+
Notes: 
-  msgpack: Variation is ok, as long 

## Optional - Adding Prefect Cloud

<center><img src="../assets/many-machine/f4.png" alt="Drawing" style="width: 600px;"/></center>


```shell
$ prefect cloud workspace set --workspace "adamgreenadgefficiencycom/kiwipycon-tutorial"
$ prefect cloud login -k $YOUR_PREFECT_API_KEY
```


## Exercise

1. Setup this Dask/Coiled stack on an EC2 cluster,
2. Add Prefect Cloud.